<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/495_EPOv2_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""Test Phase 2.2: Data Loading Node

Tests for the data loading node - tests the orchestration of data loading utilities.
"""

import sys
from pathlib import Path

# Add project root to path
project_root = Path(__file__).parent
sys.path.insert(0, str(project_root))

from agents.epo.nodes import data_loading_node
from config import ExperimentationPortfolioOrchestratorState, ExperimentationPortfolioOrchestratorConfig


def test_data_loading_node_portfolio_wide():
    """Test data loading node for portfolio-wide analysis"""
    state: ExperimentationPortfolioOrchestratorState = {
        "experiment_id": None,
        "errors": []
    }
    config = ExperimentationPortfolioOrchestratorConfig()

    result = data_loading_node(state, config)

    assert "portfolio" in result
    assert "experiment_definitions" in result
    assert "experiment_metrics" in result
    assert "experiment_analysis" in result
    assert "experiment_decisions" in result
    assert "experiment_learnings" in result
    assert "experiment_audit_log" in result

    # Check lookups
    assert "portfolio_lookup" in result
    assert "definitions_lookup" in result
    assert "metrics_lookup" in result
    assert "analysis_lookup" in result
    assert "decisions_lookup" in result
    assert "learnings_lookup" in result
    assert "audit_log_lookup" in result

    # Check data loaded
    assert len(result["portfolio"]) > 0
    assert len(result["experiment_definitions"]) > 0
    assert len(result["experiment_metrics"]) > 0

    # Check lookups work
    assert "E001" in result["portfolio_lookup"]
    assert "E001" in result["definitions_lookup"]
    assert "E001" in result["metrics_lookup"]

    assert len(result.get("errors", [])) == 0

    print("✅ test_data_loading_node_portfolio_wide passed")


def test_data_loading_node_single_experiment():
    """Test data loading node for single experiment analysis"""
    state: ExperimentationPortfolioOrchestratorState = {
        "experiment_id": "E001",
        "errors": []
    }
    config = ExperimentationPortfolioOrchestratorConfig()

    result = data_loading_node(state, config)

    # All data should be filtered to E001 only
    assert len(result["portfolio"]) == 1
    assert result["portfolio"][0]["experiment_id"] == "E001"

    assert len(result["experiment_definitions"]) == 1
    assert result["experiment_definitions"][0]["experiment_id"] == "E001"

    # Metrics should have multiple entries (control + treatment variants)
    assert len(result["experiment_metrics"]) >= 2
    assert all(m["experiment_id"] == "E001" for m in result["experiment_metrics"])

    # Check lookups
    assert "E001" in result["portfolio_lookup"]
    assert "E002" not in result["portfolio_lookup"]  # Should be filtered out

    assert len(result.get("errors", [])) == 0

    print("✅ test_data_loading_node_single_experiment passed")


def test_data_loading_node_nonexistent_experiment():
    """Test data loading node with nonexistent experiment ID"""
    state: ExperimentationPortfolioOrchestratorState = {
        "experiment_id": "E999",
        "errors": []
    }
    config = ExperimentationPortfolioOrchestratorConfig()

    result = data_loading_node(state, config)

    # Should still succeed, but with empty data
    assert "portfolio" in result
    assert len(result["portfolio"]) == 0
    assert len(result["experiment_definitions"]) == 0

    # Lookups should be empty
    assert len(result["portfolio_lookup"]) == 0

    assert len(result.get("errors", [])) == 0  # No error, just empty results

    print("✅ test_data_loading_node_nonexistent_experiment passed")


def test_data_loading_node_invalid_file():
    """Test data loading node error handling for invalid file path"""
    state: ExperimentationPortfolioOrchestratorState = {
        "experiment_id": None,
        "errors": []
    }
    config = ExperimentationPortfolioOrchestratorConfig()
    config.data_dir = "nonexistent/directory"

    result = data_loading_node(state, config)

    # Should have errors
    assert len(result.get("errors", [])) > 0
    assert "data_loading_node" in result["errors"][0]

    print("✅ test_data_loading_node_invalid_file passed")


def test_data_loading_node_integration():
    """Test data loading node integrated with goal and planning"""
    # Start with goal and planning
    from agents.epo.nodes import goal_node, planning_node

    state: ExperimentationPortfolioOrchestratorState = {
        "experiment_id": "E001",
        "errors": []
    }

    # Run goal node
    state = goal_node(state)
    assert "goal" in state

    # Run planning node
    state = planning_node(state)
    assert "plan" in state

    # Run data loading node
    config = ExperimentationPortfolioOrchestratorConfig()
    state = data_loading_node(state, config)

    # Check all data loaded
    assert "portfolio" in state
    assert "experiment_definitions" in state
    assert "definitions_lookup" in state

    # Check data is filtered to E001
    assert len(state["portfolio"]) == 1
    assert state["portfolio"][0]["experiment_id"] == "E001"

    assert len(state.get("errors", [])) == 0

    print("✅ test_data_loading_node_integration passed")


if __name__ == "__main__":
    print("Testing Phase 2.2: Data Loading Node\n")

    test_data_loading_node_portfolio_wide()
    test_data_loading_node_single_experiment()
    test_data_loading_node_nonexistent_experiment()
    test_data_loading_node_invalid_file()
    test_data_loading_node_integration()

    print("\n✅ All Phase 2.2 node tests passed!")


# Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_017_EPO_2.0 % python test_epo_phase2_node.py
Testing Phase 2.2: Data Loading Node

✅ test_data_loading_node_portfolio_wide passed
✅ test_data_loading_node_single_experiment passed
✅ test_data_loading_node_nonexistent_experiment passed
✅ test_data_loading_node_invalid_file passed
✅ test_data_loading_node_integration passed

✅ All Phase 2.2 node tests passed!